<a href="https://colab.research.google.com/github/rar8393/NLG_tests/blob/main/legal_roberta_model_finetune_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb

In [1]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 4.7 MB/s 
     |████████████████████████████████| 4.9 MB 73.5 MB/s 
     |████████████████████████████████| 163 kB 80.8 MB/s 
     |████████████████████████████████| 115 kB 85.1 MB/s 
     |████████████████████████████████| 212 kB 85.6 MB/s 
     |████████████████████████████████| 127 kB 88.0 MB/s 
     |████████████████████████████████| 6.6 MB 54.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import transformers

print(transformers.__version__)

4.22.2


In [6]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": 'train.txt', "validation": 'val.txt'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-44c31640dbaa9aee/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

In [9]:
#"facebook/opt-125m". "saibo/legal-roberta-base" "roberta-base"
model_checkpoint = "saibo/legal-roberta-base"

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [12]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2429 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (955 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1233 > 512). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1061 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (669 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1317 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (694 > 512). Running this sequence through the model will result in indexing errors


In [13]:
# block_size = tokenizer.model_max_length
block_size = 512

In [14]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [15]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
from transformers import AutoModelForCausalLM
#If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, is_decoder=True)

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [17]:
from transformers import Trainer, TrainingArguments

In [18]:
#https://stackoverflow.com/questions/72680734/huggingface-trainer-only-doing-3-epochs-no-matter-the-trainingarguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-health-test-1",
    evaluation_strategy = "steps", #epoch
    num_train_epochs=25, 
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    warmup_steps=1000,                # number of warmup steps for learning rate  
    save_steps=2000,
    eval_steps=500,
    save_total_limit=10,
    load_best_model_at_end= True,
    #output_dir='./results',
)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [20]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2134
  Num Epochs = 25
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6675


Step,Training Loss,Validation Loss
500,6.340300,4.287693
1000,4.131400,3.767369
1500,3.680700,3.557034
2000,3.416500,3.462902
2500,3.262600,3.398690
3000,3.128500,3.366778
3500,3.038600,3.341993
4000,2.960100,3.321561
4500,2.896000,3.316531
5000,2.848400,3.309566


***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
Saving model checkpoint to legal-roberta-base-health-test-1/checkpoint-2000
Configuration saved in legal-roberta-base-health-test-1/checkpoint-2000/config.json
Model weights saved in legal-roberta-base-health-test-1/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
Saving model checkpoint to legal-roberta-base-health-test-1/checkpoint-4000
Configuration saved in legal-roberta-base-health-test-1/checkpoint-4000/config.json
Model weights saved in legal-r

TrainOutput(global_step=6675, training_loss=3.372572501536166, metrics={'train_runtime': 2062.4107, 'train_samples_per_second': 25.868, 'train_steps_per_second': 3.237, 'total_flos': 1.40452124668416e+16, 'train_loss': 3.372572501536166, 'epoch': 25.0})

In [22]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("legal-roberta-base-health-test-1/checkpoint-6000")

loading configuration file legal-roberta-base-health-test-1/checkpoint-6000/config.json
Model config RobertaConfig {
  "_name_or_path": "legal-roberta-base-health-test-1/checkpoint-6000",
  "architectures": [
    "RobertaForCausalLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": true,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "total_flos": 2733585826172737536,
  "transformers_version": "4.22.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file legal-roberta-base-health-test-1/checkpoint-6000/pytorch_mode

In [23]:
import importlib
import generate_txt
importlib.reload(generate_txt)
generator = generate_txt.generate_model_results(tokenizer, model)

In [24]:
test_metrics_1 = generator.generate_sample("The event has a ", visualize=False)

/content/generated_text_metrics.py:52: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity.append(gen_doc.similarity(sent))
/content/generated_text_metrics.py:54: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one 

In [25]:
for key in test_metrics_1.keys():
  print(key)

The event has a 
nucleosis, the researchers explain. A rare type of Nucleioses is known as the nucleosome in the brains of children under 12 months old. They explain that scientists have not studied the significance of the Nuclein-based defense mechanism in people aged 6 years and older with any underlying condition.
 and the clinical manifestations (when diagnosed-without or without treatment), is “ not a clinical diagnosis. It is either a valid diagnosis, or it can be changed despite the diagnosis. In clinical studies, patients with a sudden onset of respiratory distress syndrome (COPS). In the emergency room, a doctor with the
 known as infection, with a range of causes affecting one of the cells. Although it is not important to seek emergency medical attention, the general warning is that anyone who is concerned should seek medical attention immediately. If a person is having an allergic reaction to food, such as onions, beer, bread, candy, breads
imalimal number, a person should s

In [26]:
for key in test_metrics_1.keys():
  print(test_metrics_1[key]['metric'])

{'number_ents': 0, 'ents': [], 'number_nouns': 1, 'nouns': ['event'], 'number_sents': 1, 'avg_sent_len': 4.0, 'sent_repeat_score': [0.0]}
{'number_ents': 4, 'ents': [Nucleioses, under 12 months old, Nuclein, 6 years], 'number_nouns': 16, 'nouns': ['nucleosis', 'researcher', 'type', 'Nucleioses', 'nucleosome', 'brain', 'child', 'month', 'scientist', 'significance', 'Nuclein', 'defense', 'mechanism', 'people', 'year', 'condition'], 'number_sents': 3, 'avg_sent_len': 18.0, 'sent_repeat_score': [0.0, 0.09999999999999998, 0.0357142857142857]}
{'number_ents': 0, 'ents': [], 'number_nouns': 15, 'nouns': ['manifestation', 'without', 'treatment', 'diagnosis', 'diagnosis', 'diagnosis', 'study', 'patient', 'onset', 'distress', 'syndrome', 'COPS', 'emergency', 'room', 'doctor'], 'number_sents': 4, 'avg_sent_len': 16.0, 'sent_repeat_score': [0.09090909090909094, 0.1875, 0.0, 0.11111111111111116]}
{'number_ents': 1, 'ents': [one], 'number_nouns': 15, 'nouns': ['infection', 'range', 'cause', 'cell', 

In [27]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_1.keys():
  print(test_metrics_1[key]['metric']['number_ents'], test_metrics_1[key]['metric']['number_nouns'],
        test_metrics_1[key]['metric']['number_sents'],  test_metrics_1[key]['metric']['avg_sent_len'],
        test_metrics_1[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
0 1 1 4.0 [0.0]
4 16 3 18.0 [0.0, 0.09999999999999998, 0.0357142857142857]
0 15 4 16.0 [0.09090909090909094, 0.1875, 0.0, 0.11111111111111116]
1 15 4 16.0 [0.0, 0.06666666666666665, 0.1923076923076923, 0.2272727272727273]
5 19 3 20.0 [0.18181818181818177, 0.1428571428571429, 0.0]
1 15 3 17.333333333333332 [0.0, 0.1071428571428571, 0.13043478260869568]


In [28]:
test_metrics_2 = generator.generate_sample("The conference started with a ", visualize=False)

In [29]:
for key in test_metrics_2.keys():
  print(key)

The conference started with a 
20% credible RT-PCR result on a logisticistic model using an overall sample of 0.8% individuals with the same symptoms or the incidence of COVID-19. In this analysis we found that most people reported having an S1P/IgO in the past 4 days after or between 12
US number. If the patient begins getting better treatments at bed, a person should always talk to their doctor, who may recommend a follow-up appointment. For general, a doctor can talk to a variety of factors, including: the date of an injury and the quality of support that the person was receiving, whether the
4 of the main logistic system and is the main focus of all the ongoing COVID-19 cases.
The conference started with a. The team then added the SIR model. The new model was applied to the scenario of a COVID-19 pandemic in Israel, and it used to perform a single test of the simulations. The model used a real-time transmission model to analyze the infection rate of a group of large numbers. A seco

In [30]:
for key in test_metrics_2.keys():
  print(test_metrics_2[key]['metric'])

{'number_ents': 0, 'ents': [], 'number_nouns': 1, 'nouns': ['conference'], 'number_sents': 1, 'avg_sent_len': 5.0, 'sent_repeat_score': [0.0]}
{'number_ents': 5, 'ents': [20%, RT-PCR, 0.8%, S1P, the past 4 days], 'number_nouns': 16, 'nouns': ['%', 'RT', 'PCR', 'result', 'model', 'sample', '%', 'individual', 'symptom', 'incidence', 'covid-19', 'analysis', 'people', 'S1P', 'IgO', 'day'], 'number_sents': 2, 'avg_sent_len': 26.0, 'sent_repeat_score': [0.10344827586206895, 0.04347826086956519]}
{'number_ents': 1, 'ents': [US], 'number_nouns': 16, 'nouns': ['US', 'number', 'patient', 'treatment', 'bed', 'person', 'doctor', 'appointment', 'doctor', 'variety', 'factor', 'date', 'injury', 'quality', 'support', 'person'], 'number_sents': 3, 'avg_sent_len': 21.333333333333332, 'sent_repeat_score': [0.0, 0.0714285714285714, 0.24242424242424243]}
{'number_ents': 1, 'ents': [4], 'number_nouns': 4, 'nouns': ['system', 'focus', 'covid-19', 'case'], 'number_sents': 1, 'avg_sent_len': 18.0, 'sent_repeat

In [31]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_2.keys():
  print(test_metrics_2[key]['metric']['number_ents'], test_metrics_2[key]['metric']['number_nouns'],
        test_metrics_2[key]['metric']['number_sents'],  test_metrics_2[key]['metric']['avg_sent_len'],
        test_metrics_2[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
0 1 1 5.0 [0.0]
5 16 2 26.0 [0.10344827586206895, 0.04347826086956519]
1 16 3 21.333333333333332 [0.0, 0.0714285714285714, 0.24242424242424243]
1 4 1 18.0 [0.2222222222222222]
3 19 5 12.6 [0.0, 0.125, 0.18518518518518523, 0.19047619047619047, 0.0]
1 15 2 27.5 [0.13157894736842102, 0.11764705882352944]


In [32]:
#event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '


In [33]:
test_metrics_3 = generator.generate_sample(sick_txt, visualize=False)

In [34]:
for key in test_metrics_3.keys():
  print(key)

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
It is important to note the fact that there are no specific vaccines used. There is evidence that vaccinations are not used during the outbreak of a child outbreak against a child with a positive test. Some vaccine alternatives include: immunosuppressant or non-vaccinated vaccines, immunosuppressant (other types
The current outbreak of COVID-19 caused by the novel coronavirus (COVID-19) caused by SARS-CoV-2 continues to threaten the globe in response to social distancing intervention and containment strategies. A national population-wide increase in social distancing and new infections followed by re-
The COVID-19 pandemic is challenging and requires immediate responses to mitigate virus spread; a new outbreak describes the dynamics of disease progression via SARS-CoV-2. There are no available vaccines or therapeutic targets. These vaccines target anti-SARS-CoV-2 antigens 

In [35]:
for key in test_metrics_3.keys():
  print(test_metrics_3[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 14, 'nouns': ['fact', 'vaccine', 'evidence', 'vaccination', 'outbreak', 'child', 'outbreak', 'child', 'test', 'vaccine', 'alternative', 'vaccine', 'immunosuppressant', 'type'], 'number_sents': 3, 'avg_sent_len': 18.0, 'sent_repeat_score': [0.06666666666666665, 0.21739130434782605, 0.125]}
{'number_ents': 0, 'ents': [], 'number_nouns': 17, 'nouns': ['outbreak', 'covid-19', 'coronavirus', 'COVID-19', 'SARS', 'CoV-2', 'globe', 'response', 'distancing', 'intervention', 'containment', 'strategy', 'population', 'increase', 'distancing', 'infection', 're-'], 'number_sents': 2, 'avg_sent_len': 24.0, 'sent_repeat_score': [0.1515151515151515, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 16, 'nouns': ['covid-19', 'response', 'virus', 'spread', 'outbreak', 'd

In [36]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_3.keys():
  print(test_metrics_3[key]['metric']['number_ents'], test_metrics_3[key]['metric']['number_nouns'],
        test_metrics_3[key]['metric']['number_sents'],  test_metrics_3[key]['metric']['avg_sent_len'],
        test_metrics_3[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
0 14 3 18.0 [0.06666666666666665, 0.21739130434782605, 0.125]
0 17 2 24.0 [0.1515151515151515, 0.0]
0 16 3 16.0 [0.0357142857142857, 0.0, 0.09090909090909094]
3 14 3 17.666666666666668 [0.12, 0.0714285714285714, 0.2142857142857143]
1 16 4 14.25 [0.0, 0.0, 0.0, 0.0]


In [37]:
event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '
#generator2(event_txt + sick_txt)
combo = event_txt + sick_txt

In [38]:
test_metrics_4 = generator.generate_sample(combo, visualize=False)

In [39]:
for key in test_metrics_4.keys():
  print(key)

Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
In the current pandemic, the COVID-19 outbreak in Wuhan, China ( China) has rapidly spread since December 2019 in China. At present, there are no effective therapeutics known to help treat COVID-19. The aim of this study is to design a vaccine, testing, and treatment
A group of scientists have reported the first study of its use of a cold drug against a human virus such as Mycobacterium tuberculosis tuberculosis (M tuberculosis). This bacterium is a viral molecule that is composed of the proteins that help regulate circulation in humans. It binds

In [40]:
for key in test_metrics_4.keys():
  print(test_metrics_4[key]['metric'])

{'number_ents': 2, 'ents': [three, this year], 'number_nouns': 23, 'nouns': ['Climate', 'Warriors', 'youth', 'Climate', 'Justice', 'series', 'webinar', 'climate', 'justice', 'initiative', 'community', 'organizer', 'movement', 'community', 'way', 'legislator', 'charge', 'climate', 'justice', 'flu', 'year', 'conference', 'vaccine'], 'number_sents': 6, 'avg_sent_len': 12.333333333333334, 'sent_repeat_score': [0.0, 0.0, 0.04347826086956519, 0.0, 0.0, 0.0]}
{'number_ents': 5, 'ents': [Wuhan, China, China, December 2019, China], 'number_nouns': 16, 'nouns': ['pandemic', 'covid-19', 'outbreak', 'Wuhan', 'China', 'China', 'December', 'China', 'present', 'therapeutic', 'covid-19', 'aim', 'study', 'vaccine', 'testing', 'treatment'], 'number_sents': 3, 'avg_sent_len': 17.666666666666668, 'sent_repeat_score': [0.25, 0.0, 0.06666666666666665]}
{'number_ents': 1, 'ents': [first], 'number_nouns': 19, 'nouns': ['group', 'scientist', 'study', 'use', 'drug', 'virus', 'Mycobacterium', 'tuberculosis', 'tu

In [41]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_4.keys():
  metric_load = test_metrics_4[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
2 23 6 12.333333333333334 [0.0, 0.0, 0.04347826086956519, 0.0, 0.0, 0.0]
5 16 3 17.666666666666668 [0.25, 0.0, 0.06666666666666665]
1 19 3 19.666666666666668 [0.19999999999999996, 0.10526315789473684, 0.09999999999999998]
3 20 3 18.666666666666668 [0.0, 0.20588235294117652, 0.0]
2 16 3 18.0 [0.11111111111111116, 0.0, 0.0]
1 10 3 20.333333333333332 [0.06666666666666665, 0.08695652173913049, 0.08695652173913049]


In [42]:
#check different lengths
length_metric_1 = generator.generate_sample(sick_txt, max_length=32, visualize=False)
length_metric_2 = generator.generate_sample(sick_txt, max_length=64, visualize=False)
length_metric_3 = generator.generate_sample(sick_txt, max_length=128, visualize=False)

In [43]:
for key in length_metric_1.keys():
  print(key)
  print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
The COVID-19 pandemic is the epicenter of an unprecedented pandemic. SARS-CoV-2 is the primary cause of death in
112
The COVID-19 pandemic threatens to overwhelm health care system around the globe. Our study investigates the effects of contact tracing on the degree of susceptibility to
170
It is also important to note that the most convenient method of collecting influenza vaccination data depends on how much it contains in the database. When using the vaccine, the
178
The COVID-19 pandemic provides a short and uncertain and reliable tool to guide public health efforts to control the SARS-CoV-2 pand
132
SARS-CoV-2 (COVID-19) originated in Wuhan and became a global pandemic since March 15 and spread to other
105


In [44]:
for key in length_metric_1.keys():
  print(length_metric_1[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 7, 'nouns': ['covid-19', 'epicenter', 'pandemic', 'SARS', 'CoV-2', 'cause', 'death'], 'number_sents': 2, 'avg_sent_len': 10.5, 'sent_repeat_score': [0.18181818181818177, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 11, 'nouns': ['covid-19', 'pandemic', 'health', 'care', 'system', 'globe', 'study', 'effect', 'contact', 'degree', 'susceptibility'], 'number_sents': 2, 'avg_sent_len': 13.5, 'sent_repeat_score': [0.07692307692307687, 0.1428571428571429]}
{'number_ents': 0, 'ents': [], 'number_nouns': 6, 'nouns': ['method', 'influenza', 'vaccination', 'datum', 'database', 'vaccine'], 'number_sents': 2, 'avg_sent_len': 16.0, 'sent_repeat_score': [0.07692307692307687, 0.16666666666666663]}
{'number_ents': 0, 'ents': [], 'number_nouns': 8, 'nouns': ['covid

In [45]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_1.keys():
  metric_load = length_metric_1[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
0 7 2 10.5 [0.18181818181818177, 0.0]
0 11 2 13.5 [0.07692307692307687, 0.1428571428571429]
0 6 2 16.0 [0.07692307692307687, 0.16666666666666663]
0 8 1 23.0 [0.13043478260869568]
2 6 1 21.0 [0.04761904761904767]


In [46]:
for key in length_metric_2.keys():
  print(key)

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
Coronavirus disease 2019 (COVID-19) is an infectious disease with a substantial mortality. In a clinical trial, a team examined the lymphocytes and lymph nodes in 597 people during their 20s and 40s. This trial investigated the cytokines and cytokines in COVID-19. The
Many people may have flu-like symptoms, and the exact cause is unknown. However, there is good chance for them.
There are two main main phases: the early phase of the emergency health response, which is the phase during which the COVID-19 epidemic peaks. A first stage may result in a positive outcome given the age of the children, which is the standard. This phase may be the second stage, since this stage occurs
The Spanish government has declared a lockdown for two weeks or more. In this March to determine the optimal course of action of this lockdown, we propose a framework for understanding how to provide 

In [47]:
for key in length_metric_2.keys():
    print(key)
    print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
Coronavirus disease 2019 (COVID-19) is an infectious disease with a substantial mortality. In a clinical trial, a team examined the lymphocytes and lymph nodes in 597 people during their 20s and 40s. This trial investigated the cytokines and cytokines in COVID-19. The
268
Many people may have flu-like symptoms, and the exact cause is unknown. However, there is good chance for them.
111
There are two main main phases: the early phase of the emergency health response, which is the phase during which the COVID-19 epidemic peaks. A first stage may result in a positive outcome given the age of the children, which is the standard. This phase may be the second stage, since this stage occurs
303
The Spanish government has declared a lockdown for two weeks or more. In this March to determine the optimal course of action of this lockdown, we propose a framework for understanding

In [48]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_2.keys():
  metric_load = length_metric_2[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
3 17 4 12.0 [0.06666666666666665, 0.13636363636363635, 0.09999999999999998, 0.0]
0 5 2 12.5 [0.0, 0.0]
3 17 3 20.0 [0.2962962962962963, 0.1428571428571429, 0.16666666666666663]
3 14 3 21.0 [0.0, 0.1333333333333333, 0.09999999999999998]
1 13 2 23.0 [0.0, 0.25]


In [49]:
for key in length_metric_3.keys():
  print(key)
  print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114

0
Objectives: To explore the role of air pollution on the health of various countries. To examine the role of air pollution in the mortality of COVID-19 patients across the globe. Methods: Data on aerosol aerosolation aerosols (from Air Quality Quality Testing) was used to test quality of air samples from a total of 6,500 COVID-19 patients in Italy. These samples were collected at the Center for Disease Control and Prevention (CDC), either from a hospital, or in a state where some pollutants are present in the atmosphere. In the Centers for Disease Control and Prevention (CDC), air pollution from respiratory illnesses
623
If you buy something through a link on this page, we may earn a small commission. How this works. In some cases, the flu may be a cause of concern, but others require a stronger medical diagnosis. The virus will soon leave the body once there is a ch

In [50]:
for key in length_metric_3.keys():
  print(length_metric_3[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 0, 'nouns': [], 'number_sents': 0, 'avg_sent_len': 0.0, 'sent_repeat_score': []}
{'number_ents': 7, 'ents': [Air Quality Quality Testing, 6,500, Italy, the Center for Disease Control and Prevention, CDC, the Centers for Disease Control and Prevention, CDC], 'number_nouns': 46, 'nouns': ['objective', 'role', 'air', 'pollution', 'health', 'country', 'role', 'air', 'pollution', 'mortality', 'covid-19', 'patient', 'globe', 'method', 'datum', 'aerosolation', 'aerosol', 'Air', 'Quality', 'Quality', 'Testing', 'quality', 'air', 'sample', 'total', 'covid-19', 'patient', 'Italy', 'sample', 'Center', 'Disease', 'Control', 'Prevention', 'CDC', 'hospital', 'state', 'pollutant', 'atmosphere', 'Centers', 'Disease', 'Control', 'Prevention', 'CDC', 'air', 'pollution', 'i

In [51]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_3.keys():
  metric_load = length_metric_3[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
0 0 0 0.0 []
7 46 5 23.2 [0.125, 0.17647058823529416, 0.125, 0.1470588235294118, 0.0]
1 25 7 17.285714285714285 [0.05555555555555558, 0.0, 0.09523809523809523, 0.125, 0.25, 0.0, 0.05555555555555558]
9 30 5 23.4 [0.19444444444444442, 0.09999999999999998, 0.3157894736842105, 0.09523809523809523, 0.08333333333333337]
4 28 5 21.4 [0.2272727272727273, 0.0625, 0.11111111111111116, 0.1333333333333333, 0.0]


In [ ]:
#https://huggingface.co/docs/transformers/v4.16.2/en/custom_datasets

In [ ]:
!cp -r roberta-base-health-test-1/checkpoint-12000/ roberta-health-test-v1/

In [ ]:
!zip -r roberta_health_v1.zip roberta-health-test-v1/

  adding: roberta-health-test-v1/ (stored 0%)
  adding: roberta-health-test-v1/checkpoint-12000/ (stored 0%)
  adding: roberta-health-test-v1/checkpoint-12000/rng_state.pth (deflated 27%)
  adding: roberta-health-test-v1/checkpoint-12000/pytorch_model.bin (deflated 7%)
  adding: roberta-health-test-v1/checkpoint-12000/scheduler.pt (deflated 49%)
  adding: roberta-health-test-v1/checkpoint-12000/trainer_state.json (deflated 82%)
  adding: roberta-health-test-v1/checkpoint-12000/config.json (deflated 50%)
  adding: roberta-health-test-v1/checkpoint-12000/training_args.bin (deflated 48%)
  adding: roberta-health-test-v1/checkpoint-12000/optimizer.pt (deflated 8%)


In [ ]:
from google.colab import files
files.download("roberta_health_v1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>